In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
# Load pre-trained ResNet 50 on imagenet
import tensorflow as tf
from tensorflow import keras

#IMAGE_SIZE = 255
#IMG_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)
res_net = tf.keras.applications.ResNet50(weights='imagenet')
res_net.trainable = False
# res_net.summary()

102973440/102967424 [==============================] - 1s 0us/step


In [ ]:
# Load pre-trained ResNet 152 on imagenet
import tensorflow as tf
from tensorflow import keras

IMAGE_SIZE = 255
IMG_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)
res_net = tf.keras.applications.ResNet152(weights='imagenet')
res_net.trainable = False
# res_net.summary()

242900992/242900224 [==============================] - 4s 0us/step


In [ ]:
# Removing last layer of resnet and using feature extractions
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten

output = res_net.layers[-2].output
pretrained_res_net = Model(res_net.input, outputs=output)

# pretrained_res_net.summary()

In [ ]:
# copy remaining video files to this VM space
import numpy as np
import glob
import os
from tqdm import tqdm

saved_feature_root = '/content/drive/My Drive/MasterThesis/Datasets/Youtube2Text'
npz_files = glob.glob(os.path.join(saved_feature_root, '*.npz'))
processed_video_ids = []
npz_file_count = 0
for npz_file in tqdm(npz_files):
  npz_file_count += 1
  data = np.load(npz_file)
  video_ids = data['arr_1']
  processed_video_ids.extend(video_ids)

print(len(processed_video_ids))
datasetPath = '/content/drive/My Drive/MasterThesis/Datasets/Youtube2Text/Frames'
allVideoDirectories = [f.path for f in os.scandir(datasetPath) if f.is_dir()]

count_out = 0
count_in = 0
final_list = []
for videoDir in tqdm(allVideoDirectories):
  videoId = videoDir.split('/')[-1]
  if videoId not in processed_video_ids:
    final_list.append(videoDir)

print(len(final_list))
print(npz_file_count)

100%|██████████| 1969/1969 [00:00<00:00, 51027.11it/s]

1979
40
51


In [ ]:
# Keras preprocess image input
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input
import numpy as np
import os
from tqdm import tqdm

def preprocessImageWithKeras(imagePath):
  img = image.load_img(imagePath, target_size=(224, 224))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  x = preprocess_input(x)
  return x

def extractFeaturesForFrame(frameImage):
  frame_features = pretrained_res_net.predict(frameImage)
  frame_features_flatten = frame_features.flatten()
  return frame_features_flatten

def extractFeaturesForVideo(videoFramesFolderPath):
  frameFiles = [f for f in os.listdir(videoFramesFolderPath) if os.path.isfile(os.path.join(videoFramesFolderPath, f))]
  videoFeatures = []
  for frameFile in frameFiles:
    frameFilePath = videoFramesFolderPath + '/' + frameFile
    frameOutput = preprocessImageWithKeras(frameFilePath)
    frameFeatures = extractFeaturesForFrame(frameOutput)
    videoFeatures.append(frameFeatures)
  print('video feature extracted')
  return np.array(videoFeatures)

datasetPath = '/content/drive/My Drive/MasterThesis/Datasets/Youtube2Text/Frames'
allVideoDirectories = [f.path for f in os.scandir(datasetPath) if f.is_dir()]
pretrainedFeatures = []
pretrainedVideos = []

count = 0
fileCount = npz_file_count + 1
for videoDir in tqdm(final_list):
  count = count + 1
  videoId = videoDir.split('/')[-1]
  extractedVideoFeatures = extractFeaturesForVideo(videoDir)
  pretrainedFeatures.append(extractedVideoFeatures)
  pretrainedVideos.append(videoId)
  print(len(pretrainedFeatures))
  print('******')
  if len(pretrainedFeatures) is 40:
    fileCount = fileCount + 1
    count = 0
    np.savez('/content/drive/My Drive/MasterThesis/Datasets/Youtube2Text/features_resnet50_last_'+str(fileCount), pretrainedFeatures, pretrainedVideos)
    pretrainedFeatures = []
    pretrainedVideos = []
    print('Write to file successfully!')


#pretrainedFeatures_np = np.array(pretrainedFeatures)
#pretrainedVideos_np = np.array(pretrainedVideos)
print(len(pretrainedFeatures))
#print(pretrainedFeatures_np[0].shape)
np.savez('/content/drive/My Drive/MasterThesis/Datasets/Youtube2Text/features_resnet50_'+str(fileCount), pretrainedFeatures, pretrainedVideos)





  0%|          | 0/40 [00:00<?, ?it/s]



  2%|▎         | 1/40 [00:04<02:48,  4.31s/it]

video feature extracted
1
******






  5%|▌         | 2/40 [00:14<03:48,  6.01s/it]

video feature extracted
2
******






  8%|▊         | 3/40 [00:25<04:35,  7.45s/it]

...

video feature extracted
38
******






 98%|█████████▊| 39/40 [05:57<00:12, 12.03s/it]

video feature extracted
39
******
video feature extracted
40
******






100%|██████████| 40/40 [06:02<00:00,  9.07s/it]

Write to file successfully!
0


In [ ]:
print(len(pretrainedFeatures))
print(np.array(pretrainedFeatures).shape)
print(pretrainedFeatures[0].shape)
# pretrainedFeatures_list = [np.array(f1) for f1 in pretrainedFeatures]
# pretrainedFeatures_np = np.array(pretrainedFeatures_list)
# print(pretrainedFeatures_np[0].shape)

NameError: ignored

In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

''